In [25]:
import torch
torch.__version__

'1.0.0'

In [26]:
import torch
import torchvision as tv
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as fxnl
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

In [27]:
gpu_device = torch.device("cuda:0")

# Transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))])


In [28]:
# Datasets
train_set = tv.datasets.FashionMNIST('./dataset', download=True, train=True, transform=transform)
test_set = tv.datasets.FashionMNIST('./dataset', download=True, train=False, transform=transform)

0%|          | 0/26421880 [00:00<?, ?it/s]Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to ./dataset\FashionMNIST\raw\train-images-idx3-ubyte.gz
26427392it [00:01, 23120274.76it/s]
Extracting ./dataset\FashionMNIST\raw\train-images-idx3-ubyte.gz
32768it [00:00, 295806.20it/s]
0it [00:00, ?it/s]Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz to ./dataset\FashionMNIST\raw\train-labels-idx1-ubyte.gz
Extracting ./dataset\FashionMNIST\raw\train-labels-idx1-ubyte.gz
4423680it [00:00, 12050019.63it/s]
0it [00:00, ?it/s]Extracting ./dataset\FashionMNIST\raw\t10k-images-idx3-ubyte.gz
8192it [00:00, 104956.89it/s]
Extracting ./dataset\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [29]:
# Data Loaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False, num_workers=2)

# Classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [30]:
# Helper fxn to show image
def display_image(image, one_channel=False):
    if one_channel:
        image = image.mean(dim=0)
    image = image / 2 +0.5
    np_image = image.numpy()
    if one_channel:
        plt.imshow(np_image, cmap="Greys")
    else:
        plt.imshow(np.transpose(np_image, (1, 2, 0)))

In [31]:
# Classifier Model
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.conv_layer_one = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pooling_layer_one = nn.MaxPool2d(2, 2)
        self.conv_leyer_two = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.flatten_layer_one = nn.Linear(16 * 4 * 4, 120)
        self.flatten_layer_two = nn.Linear(120, 84)
        self.flatten_layer_three = nn.Linear(84, 10)
    
    def forward(self, input_tensor: torch.Tensor):
        input_tensor = self.pooling_layer_one(fxnl.relu(self.conv_layer_one(input_tensor)))
        input_tensor = self.pooling_layer_one(fxnl.relu(self.conv_leyer_two(input_tensor)))
        input_tensor = input_tensor(-1, 16 * 4 * 4)
        input_tensor = fxnl.relu(self.flatten_layer_one(input_tensor))
        input_tensor = fxnl.relu(self.flatten_layer_two(input_tensor))
        input_tensor = self.flatten_layer_three(input_tensor)

        return input_tensor

In [32]:
neural_net = NeuralNet().to(gpu_device)
loss_function = nn.CrossEntropyLoss()
optimizer_function = optim.SGD(neural_net.parameters(), lr=0.001, momentum=0.9)

In [34]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/fashion_mnist_experiment_1')

ModuleNotFoundError: No module named 'torch.utils.tensorboard'